### Import All Necessary Liabraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tkinter as tk 
import cv2
import PIL.Image, PIL.ImageTk
import speech_recognition as sr
import roman
import datetime
import pyttsx3
import os
import math
import cv2
import time
import xlsxwriter
import xlrd
from tkinter import *
from PIL import Image
from IPython.display import clear_output
from functools import partial

### All Image Precessing Functions 

In [2]:
def plt_show(image, title=""):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.title(title)
    plt.imshow(image,cmap="Greys_r")
    plt.show()
    
class FaceDetector(object):
    def __init__(self,xml_path):
        self.classifier = cv2.CascadeClassifier(xml_path)
    
    def detect(self, image , biggest_only = True):
        scale_factor = 1.2
        min_neighbors = 5
        min_size = (30,30)
        biggest_only = True

        flags = cv2.CASCADE_FIND_BIGGEST_OBJECT | cv2.CASCADE_DO_ROUGH_SEARCH if biggest_only else cv2.CASCADE_SCALE_IMAGE
        faces_coord = self.classifier.detectMultiScale(image,scaleFactor = scale_factor,minNeighbors = min_neighbors,minSize = min_size, flags = flags)

        return faces_coord
class VideoCamera(object):
    def __init__(self,index=0):
        self.video = cv2.VideoCapture(index)
        self.index = index
        print(self.video.isOpened())
    
    def __del__(self):
        self.video.release()
        
    def get_frame(self, in_grayscle=False):
        _,frame = self.video.read()
        if in_grayscle:
            frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        return frame

def cut_faces(image, faces_coord):
    faces = []
    for (x,y,w,h) in faces_coord:
        w_rm = int(0.2 * w/2)
        faces.append(image[y:y+h,x + w_rm : x+w-w_rm])
    return faces

def normalize_intensity(images):
    images_norm = []
    for image in images:
        is_color = len(image.shape)==3
        if is_color:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images_norm.append(cv2.equalizeHist(image))
    return images_norm

def resize_image(images , size = (50,50)):
    images_norm = []
    for image in images:
        if image.shape < size:
            image_norm = cv2.resize(image, size, interpolation = cv2.INTER_AREA)
        else:
            image_norm = cv2.resize(image, size, interpolation = cv2.INTER_CUBIC)
        images_norm.append(image_norm)
        
    return images_norm

def normalize_faces(frame, face_coord):
    faces = cut_faces(frame, face_coord)
    faces = normalize_intensity(faces)
    faces = resize_image(faces)
    return faces

def draw_rect(image , coords):
    for (x,y,w,h) in coords:
        w_rm = int(0.2 * w/2)
        cv2.rectangle(image,(x+w_rm,y),(x + w - w_rm,y + h),(120,190,10),3)
    
def collect_dataset():
    images = []
    labels = []
    labels_desc = []
    Students=[]
    Present=[]
    Absent=[]
    students = [student for student in os.listdir("Students/")]
    for i,student in enumerate(students):
        labels_desc.append(student)
        Students.append(student)
        Absent.append(student)
        for image in os.listdir("Students/"+student):
            images.append(cv2.imread("Students/"+student+"/"+image,0))
            labels.append(i)
    return (images,np.array(labels),labels_desc, Present, Absent)


def build_dataset(idname):
    webcam = VideoCamera()
    detector = FaceDetector("frontal_face.xml")
    folder = "Students/" + idname
    cv2.namedWindow("Student Entry" , cv2.WINDOW_AUTOSIZE)
    if not os.path.exists(folder):
        os.mkdir(folder)
        counter = 0
        timer = 0
        while counter < 15:
            frame = webcam.get_frame()
            faces_coord = detector.detect(frame)
            if len(faces_coord) and timer % 700 == 50:
                faces = normalize_faces(frame, faces_coord)
                cv2.imwrite(folder + "/" + str(counter) + ".jpg",faces[0])
                plt_show(faces[0],"Image Saved: "+str(counter))
                clear_output(wait=True)
                counter += 1
            draw_rect(frame,faces_coord)
            cv2.imshow("Student Entry",frame)
            cv2.waitKey(50)
            timer += 50
        cv2.destroyAllWindows()
        webcam.video.release()
    else:
        print("Student Already Exist")
        
def trainModel():
    images, labels, labels_dic, Present, Absent =  collect_dataset()

    rec_eig = cv2.face.EigenFaceRecognizer_create()
    rec_eig.train(images,labels)

    rec_fisher = cv2.face.FisherFaceRecognizer_create()
    rec_fisher.train(images,labels)

    rec_lbph = cv2.face.LBPHFaceRecognizer_create()
    rec_lbph.train(images,labels)

    print("Model Trained Successfully")
    return rec_lbph

def markAtd(rec_lbph):
    webcam = VideoCamera()
    detector = FaceDetector("frontal_face.xml")
    cv2.namedWindow("Live" , cv2.WINDOW_AUTOSIZE)
    images, labels, labels_dic, Present, Absent =  collect_dataset()
    print(Present)
    print(Absent)
    while True:
        frame = webcam.get_frame()
        faces_coord = detector.detect(frame)
        if len(faces_coord):
            faces = normalize_faces(frame, faces_coord)
            for i, face in enumerate(faces):
                collector = cv2.face.StandardCollector_create()
                rec_lbph.predict_collect(face,collector)
                conf = collector.getMinDist()
                pred = collector.getMinLabel()
                threshold = 130
                print("Prediction: " + labels_dic[pred].capitalize() + ", \nConfidence: "+ str(round(conf)))
                clear_output(wait=True)
                if conf < threshold:
                    cv2.putText(frame,labels_dic[pred].capitalize(),(faces_coord[i][0], faces_coord[i][1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255),2)
                    Person=labels_dic[pred]
                    if(Person in Present):
                        continue
                    else:
                        Present.append(Person)
                        if(Person in Absent):
                            print("Remove Call")
                            Absent.remove(Person)
                else:
                    cv2.putText(frame,"Unknown",(faces_coord[i][0], faces_coord[i][1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,0,0),2)
            draw_rect(frame,faces_coord)
        cv2.putText(frame,"Esc to Exit",(5, frame.shape[0]-5),cv2.FONT_HERSHEY_PLAIN, 1.3, (255,255,255),2, cv2.LINE_AA)
        cv2.imshow("Live",frame)
        if cv2.waitKey(40) & 0xFF == 27:
            cv2.destroyAllWindows()
            break
    print(Present)
    print(Absent)
    datatofile(Present, Absent)
    del webcam

### Import Data to Excel Function

In [3]:
def datatofile(Present, Absent):
    filename = "104586/104586-" + datetime.datetime.today().strftime("%d-%m-%Y")
    if not os.path.exists(filename+".xlsx"):
        print("Not Exist")
        workbook = xlsxwriter.Workbook(filename+".xlsx")
        worksheet = workbook.add_worksheet(datetime.datetime.today().strftime("%d-%m-%Y"))
        # Increase the cell size of the merged cells to highlight the formatting.
        worksheet.set_column('A:B', 30)
        worksheet.set_row(0, 30)

        merge_format = workbook.add_format({
            'bold': 1,
            'border': 1,
            'align': 'center',
            'valign': 'vcenter',
            'fg_color': '#cccccc'})

        worksheet.merge_range('A1:B1', 'PAF-Kiet AMS Via Face-ID', merge_format)
        heading = workbook.add_format({
            'bold': 1,
            'border': 1,
            'align': 'center',
            'valign': 'vcenter'})
        worksheet.write('A2', 'Students', heading)
        worksheet.write('B2', 'Attendance', heading)

        presentstyle = workbook.add_format()
        presentstyle.set_font_color('#00a650')
        absentstyle = workbook.add_format()
        absentstyle.set_font_color('#ff0000')

        row = 2
        col = 0
        for student in Present: 
            worksheet.write(row, col, student) 
            worksheet.write(row, col + 1, "Present", presentstyle) 
            row += 1
        for student in Absent: 
            worksheet.write(row, col, student) 
            worksheet.write(row, col + 1, "Absent", absentstyle) 
            row += 1
        workbook.close()
    else:
        print("Exist")
        df = pd.read_excel(filename+".xlsx")
        header = df.iloc[0]
        df = df[1:]
        df = df.rename(columns = header)
        data = df.values
        presentold =[]
        absentold = []
        for stname, attd in data:
            if attd == "Present":
                presentold.append(stname)
            else:
                absentold.append(stname)

        print(presentold)
        print(absentold)
        for student in Present:
            if student not in presentold:
                presentold.append(student)
                if student in absentold:
                    absentold.remove(student)

        print(presentold)
        print(absentold)
        os.remove(filename+".xlsx")

        workbook = xlsxwriter.Workbook(filename+".xlsx")
        worksheet = workbook.add_worksheet(datetime.datetime.today().strftime("%d-%m-%Y"))
        # Increase the cell size of the merged cells to highlight the formatting.
        worksheet.set_column('A:B', 30)
        worksheet.set_row(0, 30)

        merge_format = workbook.add_format({
            'bold': 1,
            'border': 1,
            'align': 'center',
            'valign': 'vcenter',
            'fg_color': '#cccccc'})

        worksheet.merge_range('A1:B1', 'PAF-Kiet AMS Via Face-ID', merge_format)
        heading = workbook.add_format({
            'bold': 1,
            'border': 1,
            'align': 'center',
            'valign': 'vcenter'})
        worksheet.write('A2', 'Students', heading)
        worksheet.write('B2', 'Attendance', heading)

        presentstyle = workbook.add_format()
        presentstyle.set_font_color('#00a650')
        absentstyle = workbook.add_format()
        absentstyle.set_font_color('#ff0000')

        row = 2
        col = 0
        for student in presentold: 
            worksheet.write(row, col, student) 
            worksheet.write(row, col + 1, "Present", presentstyle) 
            row += 1
        for student in absentold: 
            worksheet.write(row, col, student) 
            worksheet.write(row, col + 1, "Absent", absentstyle) 
            row += 1
        workbook.close()  

### All Speech Assistent Functions

In [4]:
def speak(audio):
    engine = pyttsx3.init()
    engine.setProperty('voice', "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0")
    engine.say(audio)
    engine.runAndWait()

def wishme(window):
    hour = int(datetime.datetime.now().hour)
    if hour >= 0 and hour <= 12:
        var.set("Good Morning Cubers") #Name - your Name
        window.update()
        speak("Good Morning Cubers!")
    elif hour >= 12 and hour <= 18:
        var.set("Good Afternoon Cubers!")
        window.update()
        speak("Good Afternoon Cubers!")
    else:
        var.set("Good Evening Cubers")
        window.update()
        speak("Good Evening Wasim!")
    speak("Myself Edith") #BotName - Give a name to your assistant
    
def wishme2(window):
    speak("How may I help you sir") #BotName - Give a name to your assistant

def takeCommand(window):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        var.set("Listening...")
        window.update()
        print("Listening...")
        r.pause_threshold = 1
        r.energy_threshold = 300
        audio = r.listen(source)
    try:
        var.set("Recognizing...")
        window.update()
        print("Recognizing")
        query = r.recognize_google(audio, language='en-in')
    except Exception as e:
        print("Couldn't Understand")
        speak("Couldn't Understand")
        query = "None"
    window.update()
    return query

def play(window):
    stname.set("")
    model = trainModel()
    while True:
        wishme2(window)
        query = takeCommand(window).lower()
        if 'exit' in query:
            var.set("Bye sir")
            window.update()
            speak("Thank you for using AMS, Bye sir")
            window.destroy()
            break 
        elif "data set" in query:
            secondWindow(window)
            break
        elif "attendance" in query:
            var.set("Mark Attendence")
            window.update()
            speak("Here you go")
            markAtd(model)
            speak("Data import to file")
            var.set("Data import to Excel file")
            window.update()
        elif "none" not in query:
            print(query)
            speak("I cant answer that")
    

### Project UI

In [5]:
class mainWindow:
    
    def update(self, ind):
        frame = self.frames[(ind)%100]
        ind += 1
        self.label.configure(image=frame)
        self.window.after(100, self.update, ind)
    
    def __init__(self, master):
        
        self.master = master
        self.window = Frame(self.master)
        
        self.label1 = Label(self.window, textvariable = var, bg = '#ADD8E6')
        self.label1.config(font=("Courier", 20))
        var.set('Welcome')
        self.label1.pack()

        self.frames = [PhotoImage(file='Assistant.gif',format = 'gif -index %i' %(i)) for i in range(100)]        

        self.label = Label(self.window, width = 500, height = 450)
        self.label.pack()
        self.window.after(0,self.update, 0)

        self.label3 = Label(self.window, textvariable = var2, width = 20, bg = '#5C85FB', borderwidth=1, relief="groove")
        self.label3.config(font=("Courier", 14))
        var2.set('Get Data Set')
        self.label3.pack()

        self.label4 = Label(self.window, textvariable = var3, width = 20, bg = '#5C85FB', borderwidth=1, relief="groove")
        self.label4.config(font=("Courier", 14))
        var3.set('Mark Attendence')
        self.label4.pack()

        self.label5 = Label(self.window, textvariable = var4, width = 20, bg = '#FAB60C', borderwidth=1, relief="groove")
        self.label5.config(font=("Courier", 14))
        var4.set('Exit')
        self.label5.pack()
        
        self.window.pack()
        wishme(master)
        play(master)


def secondWindow(root):
    window = tk.Toplevel()
    window.wm_title("Window")
    
    abc = [root,window]
    window.bind('<Return>',lambda event, args=abc: callback(event,args))

    label2 = Label(window, textvariable = var1)
    label2.config(font=("Courier", 11))
    var1.set('Insert Student ID_Name and Press Enter')
    label2.pack()

    txtBox = Entry(window, width = 30, textvariable = stname)
    txtBox.config(font=("Courier", 12))
    txtBox.pack()

    global labelerror
    labelerror = Label(window, textvariable = txterror, fg = "red")
    labelerror.config(font=("Courier", 8))
    txterror.set('Please enter StudentID_Name like: 9323_Wasim')
    labelerror.pack_forget()
    
    wWidth = window.winfo_reqwidth()
    wHeight = window.winfo_reqheight()
    pRight = int(window.winfo_screenwidth()/2.5 - wWidth/1.2)
    pDown = int(window.winfo_screenheight()/3 - wHeight/2)
    window.geometry("+{}+{}".format(pRight, pDown))
    
        
def callback(event,abc):
    root = abc[0]
    window = abc[1]
    n = stname.get()
    if n is not "":
        folder = "Students/" + n
        if not os.path.exists(folder):
            var.set("Collecting Images")
            root.update()
            speak("Stand in front of Camera and Dont move")
            build_dataset(stname.get())
            model = trainModel()
            window.destroy()
            speak("Congratulations, Successfully Collected Your Data")
            play(root)
        else:
            txterror.set("Student Already Exist")
            labelerror.pack()
    else:
        txterror.set("Please enter StudentID_Name like: 9323_Wasim")
        labelerror.pack()

def main(): 
    global Students
    Students = []
    global Present
    Present = []
    global Absent
    Absent=[]
    window = tk.Tk()
    global var
    global var2
    global var3
    global var4
    global var1
    global stname
    global txterror

    var = StringVar()
    var2 = StringVar()
    var3 = StringVar()
    var4 = StringVar()
    var1 = StringVar()
    stname = StringVar()
    txterror = StringVar()
    
    window.title('EDITH')
    
    windowWidth = window.winfo_reqwidth()
    windowHeight = window.winfo_reqheight()

    positionRight = int(window.winfo_screenwidth()/2.5 - windowWidth/1.2)
    positionDown = int(window.winfo_screenheight()/3 - windowHeight)

    window.geometry("+{}+{}".format(positionRight, positionDown))
    app = mainWindow(window)
    window.mainloop()

In [6]:
main()

['Mustafa']
['1234_Zaniq', '9306_Waqas', '9323_Wasim', '9326_Wajahat', 'Fayyaz', 'Maaz', 'Muneeb', 'Rafay', 'Wasim9323', 'WasimAhmed']
Not Exist
Listening...
Recognizing


In [ ]:

speak("Hello")

secondWindow(window)



In [ ]:
engine = pyttsx3.init()
voices = engine.getProperty('voices')
for voice in voices:
    engine.setProperty('voice', voice.id)
    print(voice.id)
    engine.say('The quick brown fox jumped over the lazy dog.')
engine.runAndWait()